In [ ]:
# Initial things to use the drone

# from djitellopy import tello
# from time import sleep

# me = tello.Tello()
# To connect 
# me.connect()

# To takeoff
# me.takeoff()

# To control movement
# me.send_rc_control(left/right, forward/backward, up/down, yaw_velocity)

# To stop
# sleep(5)

# To land
# me.land()

Litreature 
- video of drone Face tracking part 
https://www.youtube.com/watch?v=LmEcyQnfpDA

- cv2 github repo to fetch the datasets


In [2]:
# FACE TRACKING - this works very well

import cv2
import numpy as np

# Detecting a face
def findFace(img):
    faseCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faseCascade.detectMultiScale(imgGray, 1.2, 8)

    myFaceList = []
    myFaceListArea = []

    for(x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w//2
        cy = y + h//2
        area = w*h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myFaceList.append((cx, cy, area))
        myFaceListArea.append(area)
    
    if len(myFaceList)!= 0:
        i = myFaceListArea.index(max(myFaceListArea))
        return img, [myFaceList[i], myFaceListArea[i]]
    
    # Default return to prevent errors
    return img, [[0, 0], 0]
    
# def trackFace(me, info, w, pidf, pError):
#     return


cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findFace(img)
    print(f'Area: {info[1]}, Center: {info[0]}')
    cv2.imshow("Face Tracking", img)
    cv2.waitKey(1)

    


Area: 127449, Center: (696, 454, 127449)
Area: 136900, Center: (708, 451, 136900)
Area: 131769, Center: (715, 444, 131769)
Area: 137641, Center: (721, 442, 137641)
Area: 142129, Center: (726, 437, 142129)
Area: 129600, Center: (722, 431, 129600)
Area: 133225, Center: (718, 435, 133225)
Area: 132496, Center: (723, 431, 132496)
Area: 131044, Center: (725, 386, 131044)
Area: 143641, Center: (724, 389, 143641)
Area: 129600, Center: (722, 385, 129600)
Area: 121801, Center: (708, 390, 121801)
Area: 121801, Center: (695, 392, 121801)
Area: 119025, Center: (684, 398, 119025)
Area: 116281, Center: (682, 399, 116281)
Area: 123201, Center: (682, 378, 123201)
Area: 124609, Center: (679, 386, 124609)
Area: 123904, Center: (684, 380, 123904)
Area: 149769, Center: (685, 381, 149769)
Area: 133225, Center: (682, 370, 133225)
Area: 141376, Center: (684, 367, 141376)
Area: 142884, Center: (683, 368, 142884)
Area: 133956, Center: (678, 373, 133956)
Area: 139876, Center: (679, 379, 139876)
Area: 122500, Ce

KeyboardInterrupt: 

In [1]:
# This is detecting clocks as well as people

import cv2
import numpy as np

# Detecting a person
def findPerson(img):
    personCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_upperbody.xml")
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    persons = personCascade.detectMultiScale(imgGray, 1.1, 4)  # Adjust scaleFactor & minNeighbors if needed

    myPersonList = []
    myPersonListArea = []

    for (x, y, w, h) in persons:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cx = x + w // 2
        cy = y + h // 2
        area = w * h
        cv2.circle(img, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        myPersonList.append((cx, cy))
        myPersonListArea.append(area)

    if len(myPersonList) != 0:
        i = myPersonListArea.index(max(myPersonListArea))  # Select largest detected person
        return img, [myPersonList[i], myPersonListArea[i]]

    return img, [[0, 0], 0]  # Default return to prevent errors

cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    img, info = findPerson(img)
    print(f'Area: {info[1]}, Center: {info[0]}')

    cv2.imshow("Person Detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit

cap.release()
cv2.destroyAllWindows()


Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 9222, Center: (234, 492)


2025-02-05 18:08:27.787 python[10268:221917] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-05 18:08:27.787 python[10268:221917] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Area: 14850, Center: (778, 284)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 78177, Center: (365, 395)
Area: 104244, Center: (381, 393)
Area: 3276, Center: (312, 341)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 3445, Center: (469, 152)
Area: 0, Center: [0, 0]
Area: 322792, Center: (714, 435)
Area: 2726, Center: (563, 506)
Area: 3162, Center: (564, 505)
Area: 0, Center: [0, 0]
Area: 36503, Center: (325, 377)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 3328, Center: (577, 482)
Area: 0, Center: [0, 0]
Area: 6336, Center: (565, 509)
Area: 6177, Center: (563, 509)
Area: 0, Center: [0, 0]
Area: 0, Center: [0, 0]
Area: 4380, Center: (

KeyboardInterrupt: 

Using:

- cv2 to capture the image
- Yolo to detect the person - include this in litreature review
- https://docs.ultralytics.com/models/yolov8/ 


In [1]:
# This is more accurate

import cv2
from ultralytics import YOLO

# Load YOLOv8 model
# Smallest YOLOv8 model
model = YOLO("yolov8n.pt")  

# Open webcam
cap = cv2.VideoCapture(0) 

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Inference on the frame 
    results = model(frame) 

    myPersonList = []
    myPersonListArea = []

    for r in results:
        for box in r.boxes:
            # Get class ID
            cls = int(box.cls[0]) 
            # Confidence score 
            conf = box.conf[0].item()  
            # Class 0 = "person", confidence > 80%
            if cls == 0 and conf > 0.8:  
                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0]) 
                # Center of bounding box
                cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  
                # Calculate area
                area = (x2 - x1) * (y2 - y1)  
                
                # Depending on the size of the bounding box the drone will move
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2) 
                # This is what the drone will follow
                # Draw center point
                cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED) 

                myPersonList.append((cx, cy))
                myPersonListArea.append(area)

    # Select the largest detected person
    if myPersonList:
        i = myPersonListArea.index(max(myPersonListArea))
        print(f'Area: {myPersonListArea[i]}, Center: {myPersonList[i]}')


    cv2.imshow("Person Detection (YOLOv8)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break  # Press 'q' to exit

cap.release()
cv2.destroyAllWindows()



0: 384x640 1 person, 51.6ms
Speed: 3.2ms preprocess, 51.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1 cell phone, 56.3ms
Speed: 8.9ms preprocess, 56.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 334815, Center: (776, 460)

0: 384x640 2 persons, 1 cell phone, 1 clock, 48.7ms
Speed: 1.2ms preprocess, 48.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Area: 334972, Center: (776, 460)

0: 384x640 2 persons, 1 clock, 61.2ms
Speed: 1.4ms preprocess, 61.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Area: 335634, Center: (778, 459)

0: 384x640 2 persons, 1 clock, 49.3ms
Speed: 1.4ms preprocess, 49.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Area: 333300, Center: (777, 460)

0: 384x640 2 persons, 1 clock, 65.6ms
Speed: 1.1ms preprocess, 65.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Area: 333454, Center: (776, 460)



2025-02-05 18:13:28.029 python[10483:227497] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-05 18:13:28.029 python[10483:227497] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0: 384x640 2 persons, 1 clock, 68.8ms
Speed: 1.6ms preprocess, 68.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Area: 336330, Center: (779, 460)

0: 384x640 2 persons, 1 clock, 52.1ms
Speed: 1.3ms preprocess, 52.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Area: 336141, Center: (778, 459)

0: 384x640 2 persons, 45.6ms
Speed: 1.2ms preprocess, 45.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Area: 336330, Center: (777, 460)

0: 384x640 2 persons, 1 clock, 45.0ms
Speed: 1.4ms preprocess, 45.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Area: 337340, Center: (781, 460)

0: 384x640 2 persons, 1 clock, 45.3ms
Speed: 1.0ms preprocess, 45.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 332948, Center: (778, 460)

0: 384x640 2 persons, 1 clock, 48.0ms
Speed: 1.3ms preprocess, 48.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Area: 333144, Center: (776, 4

KeyboardInterrupt: 